<a href="https://colab.research.google.com/github/dotsnangles/Transformers-Text-Classification/blob/main/%EB%89%B4%EC%8A%A4_%ED%86%A0%ED%94%BD_%EB%B6%84%EB%A5%98_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q folium
!pip install -q transformers
!pip install -q datasets
!pip install -q huggingface_hub

     |████████████████████████████████| 4.2 MB 5.4 MB/s 
     |████████████████████████████████| 86 kB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 20.9 MB/s 
     |████████████████████████████████| 596 kB 50.4 MB/s 
     |████████████████████████████████| 346 kB 5.2 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 212 kB 47.0 MB/s 
     |████████████████████████████████| 140 kB 58.2 MB/s 
     |████████████████████████████████| 1.1 MB 43.7 MB/s 
     |████████████████████████████████| 127 kB 57.7 MB/s 
     |████████████████████████████████| 271 kB 51.4 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 54.1 MB/s 
     |████████████████████████████████| 112 kB 58.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 re

In [62]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import datasets
import pandas as pd
from huggingface_hub import notebook_login
import torch
import numpy as np

In [3]:
from sklearn.metrics import accuracy_score, f1_score

In [4]:
notebook_login()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:587: FutureWarning: HfApi.login: This method is deprecated in favor of `set_access_token` and will be removed in v0.8.
  FutureWarning,


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [5]:
num_labels = 7

In [6]:
model_ckpt = 'klue/roberta-small'
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels,)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/260M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'cla

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [7]:
data = datasets.load_dataset('csv', data_files='/content/train_data.csv')['train']
data = data.train_test_split(test_size=0.2, seed=100)
data = data.remove_columns('index')

Using custom data configuration default-09fd8d7167015398


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-09fd8d7167015398/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
def tokenize(batch):
    return tokenizer(batch["title"], padding=True, truncation=True)

In [9]:
encoded_data = data.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
encoded_data = encoded_data.rename_column('topic_idx', 'label')

In [11]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [12]:
batch_size = 64
logging_steps = len(encoded_data['train']) // batch_size
model_name = f"{model_ckpt}-finetuned-klue-roberta-tc"
training_args = TrainingArguments(output_dir=model_name,
                                    num_train_epochs=2,
                                    learning_rate=2e-5,
                                    per_device_train_batch_size=batch_size,
                                    per_device_eval_batch_size=batch_size,
                                    weight_decay=0.01,
                                    evaluation_strategy="epoch",
                                    disable_tqdm=False,
                                    logging_steps=logging_steps,
                                    push_to_hub=True,
                                    log_level="error")

In [13]:
trainer = Trainer(model=model, args=training_args,
                    compute_metrics=compute_metrics,
                    train_dataset=encoded_data['train'],
                    eval_dataset=encoded_data['test'],
                    tokenizer=tokenizer)
trainer.train()

Cloning https://huggingface.co/dotsnangles/roberta-small-finetuned-klue-roberta-tc into local empty directory.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.485700,0.337246,0.887197,0.885845
2,0.293900,0.317831,0.894097,0.893411


TrainOutput(global_step=1142, training_loss=0.3895495342916031, metrics={'train_runtime': 169.7017, 'train_samples_per_second': 430.438, 'train_steps_per_second': 6.729, 'total_flos': 548115656028972.0, 'train_loss': 0.3895495342916031, 'epoch': 2.0})

In [88]:
data_test = datasets.load_dataset('csv', data_files='/content/test_data.csv')['train']

Using custom data configuration default-49dd8b7dbc5c73cf


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-49dd8b7dbc5c73cf/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [89]:
data_test = data_test.remove_columns('index')

In [90]:
encoded_test = data_test.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [104]:
preds_output = trainer.predict(encoded_test)

In [110]:
logits = preds_output.predictions

In [113]:
logits

array([[ 2.988469  ,  0.43541917,  1.7243724 , ..., -2.4054534 ,
        -2.124446  , -2.4569125 ],
       [-1.1199098 , -1.3750958 ,  0.28736368, ..., -0.8440632 ,
        -1.0654103 , -1.5288186 ],
       [ 0.6928738 ,  0.74988633,  4.061738  , ..., -1.8037657 ,
        -2.767442  ,  0.00903034],
       ...,
       [-1.6658198 , -1.5762547 ,  3.9582565 , ..., -0.8784644 ,
        -1.7239038 , -0.665548  ],
       [ 1.6542959 ,  1.1981778 ,  2.94076   , ..., -1.7791437 ,
        -2.322952  , -2.4418306 ],
       [ 1.3292501 , -0.21448615,  3.1863472 , ..., -2.1827576 ,
        -2.0787804 ,  1.8507029 ]], dtype=float32)

In [115]:
print(logits.argmax(1))

[0 3 2 ... 2 2 2]


In [ ]:
pd.Data

In [120]:
preds_output[0]

array([[ 2.988469  ,  0.43541917,  1.7243724 , ..., -2.4054534 ,
        -2.124446  , -2.4569125 ],
       [-1.1199098 , -1.3750958 ,  0.28736368, ..., -0.8440632 ,
        -1.0654103 , -1.5288186 ],
       [ 0.6928738 ,  0.74988633,  4.061738  , ..., -1.8037657 ,
        -2.767442  ,  0.00903034],
       ...,
       [-1.6658198 , -1.5762547 ,  3.9582565 , ..., -0.8784644 ,
        -1.7239038 , -0.665548  ],
       [ 1.6542959 ,  1.1981778 ,  2.94076   , ..., -1.7791437 ,
        -2.322952  , -2.4418306 ],
       [ 1.3292501 , -0.21448615,  3.1863472 , ..., -2.1827576 ,
        -2.0787804 ,  1.8507029 ]], dtype=float32)